# Preprocessing Model's NQL result

In [82]:
# python3
from openai import OpenAI

client = OpenAI(api_key="sk-8d0460b9ce124218b9b3b7794a8f7e08", base_url="https://api.deepseek.com/v1")

response = client.chat.completions.create(
    model="deepseek-coder",
    messages=[
        {"role": "system", "content": "You are a helpful assistant"},
        {"role": "user", "content": "Generate example SQL query"},
    ]
)

print(response.choices[0].message.content)


Sure, although you didn't specify a type of query, I'll make a simple example SQL query:

```
SELECT * FROM Users WHERE Age > 21;
```
This query selects all records from the "Users" table where the "Age" is greater than 21.
Please specify the type of SQL query you want to generate if you need a complex example.



In [5]:
example_text = """To answer this question, we need to know the structure of your database. However, assuming you have a table named "users" with a column named "last_active_date", you could use the following SQL query:

```
SELECT COUNT(*) 
FROM users 
WHERE last_active_date >= CURDATE() - INTERVAL 14 DAY;
```

This query will count the number of users who were active in the last 14 days. Please replace the table and column names with your actual table and column names.
"""

In [6]:
example_text = """SELECT COUNT(*) 
FROM users 
WHERE last_active_date >= CURDATE() - INTERVAL 14 DAY"""

In [7]:
import re

def extract_sql_query(text):
    
    sql_patterns = [r'```sql(.*?)```', r'```(.*?)```', r'(SELECT.*?;)', r'(SELECT.*)']
    
    for pattern in sql_patterns:
        match = re.search(pattern, text, re.DOTALL)
        if match:
            return match.group(1).strip()
    return None

extract_sql_query(example_text)

'SELECT COUNT(*) \nFROM users \nWHERE last_active_date >= CURDATE() - INTERVAL 14 DAY'

# With Experiment result

In [10]:
import re, json, os, sqlite3
import pandas as pd

In [13]:
result_file = pd.read_excel('../src/pointx/NLQ2SQL model exp result.xlsx').iloc[:,3:]
result_file.head()

,Actual SQL,Pure Question DeepSeek,Pure Question GPT3.5,Pure Question GPT4,Pure Question Gemini,Schema Description DeepSeek,Schema Description GPT3.5,Schema Description GPT4,Schema Description Gemini,ChatQ - Only NSQL,...,Pure Question Gemini result,Schema Description DeepSeek result,Schema Description GPT3.5 result,Schema Description GPT4 result,Schema Description Gemini result,ChatQ - NSQL result,ChatQ - DeepSeek result,ChatQ - GPT3.5 result,ChatQ - GPT4 result,ChatQ - Gemini result
0,"SELECT month_id, SUM(ntx_pointx_financial) FRO...","SELECT \n DATE_FORMAT(transaction_date, '%Y...","SELECT MONTH(transaction_date) AS month, SUM(a...","SELECT SUM(amount) AS total_amount, MONTH(tran...","SELECT SUM(amount) AS total_amount, STRFTIME('...","SELECT month_id, SUM(ntx_pointx_financial) FRO...","SELECT month_id, SUM(ntx_pointx_financial) FRO...","SELECT month_id, SUM(ntx_pointx_financial) FRO...","SELECT month_id, SUM(ntx_pointx_financial) AS ...","SELECT SUM(mtd1_amt_point_topup_auto_wealth), ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,SELECT SUM(amt_point_topup) FROM pointx_keymat...,SELECT SUM(points) AS total_points\nFROM top_u...,SELECT SUM(points) FROM transactions WHERE tra...,SELECT SUM(points) FROM transactions WHERE tra...,SELECT SUM(amount) FROM financial_transactions...,SELECT SUM(amt_point_topup) FROM pointx_keymat...,SELECT SUM(amt_point_topup) FROM pointx_keymat...,SELECT SUM(amt_point_topup) FROM pointx_keymat...,SELECT SUM(amt_point_topup) FROM pointx_keymat...,SELECT SUM(mtd1_amt_point_topup) FROM pointx_k...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"SELECT month_id, SUM(amt_point_pay) FROM point...","SELECT \n DATE_FORMAT(transaction_date, '%Y...","SELECT MONTH(payment_date) AS month, SUM(point...","SELECT DATE_FORMAT(payment_date, '%Y-%m') AS m...",SELECT SUM(amount) FROM financial_transactions...,"SELECT month_id, SUM(amt_point_pay) FROM point...","SELECT month_id, SUM(amt_point_pay) FROM point...","SELECT month_id, SUM(amt_point_pay) FROM point...","SELECT\n month_id,\n SUM(amt_point_pay) AS t...",SELECT SUM(mtd1_amt_point_pay) FROM pointx_key...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,SELECT AVG(rate_point_per_baht_pay) FROM point...,SELECT AVG(released_points) AS average_rate\nF...,SELECT AVG(released_points) FROM payment_custo...,SELECT AVG(rate) FROM points WHERE customer_ty...,"SELECT SUM(amount) AS total_amount, strftime('...",SELECT AVG(rate_point_per_baht_pay) FROM point...,SELECT AVG(rate_point_per_baht_pay) FROM point...,SELECT AVG(rate_point_per_baht_pay) FROM point...,SELECT rate_point_per_baht_pay FROM pointx_key...,SELECT AVG(rate_point_per_baht_pay) FROM point...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"SELECT month_id, AVG(ncust_visit) FROM pointx_...","SELECT \n DATE_FORMAT(visit_date, '%Y-%m') ...",SELECT AVG(number_of_customers)\nFROM visits\n...,SELECT AVG(monthly_visitors) FROM customer_vis...,"SELECT SUM(amount) AS ""Total Amount""\nFROM fin...","SELECT month_id, AVG(ncust_visit) \nFROM point...",SELECT AVG(ncust_visit) FROM pointx_keymatrix_...,"SELECT month_id, AVG(ncust_pointx_visit) FROM ...","SELECT \n\tmonth_id, \n\tAVG(ncust_visit) AS a...","SELECT AVG(mtd1_ncust_visit), AVG(mtd1_ncust_u...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
# Convert all elements in the DataFrame to strings
result_file = result_file.applymap(str)

for col in result_file.columns:
    result_file[col] = result_file[col].apply(extract_sql_query)

# result_file.to_excel('results/temp_prep_exp_result.xlsx', index=False)
result_file.head()

,Actual SQL,Pure Question DeepSeek,Pure Question GPT3.5,Pure Question GPT4,Pure Question Gemini,Schema Description DeepSeek,Schema Description GPT3.5,Schema Description GPT4,Schema Description Gemini,ChatQ - Only NSQL,ChatQ - DeepSeek,ChatQ - GPT3.5,ChatQ - GPT4,ChatQ - Gemini
0,"SELECT month_id, SUM(ntx_pointx_financial) FRO...","SELECT \n DATE_FORMAT(transaction_date, '%Y...","SELECT MONTH(transaction_date) AS month, SUM(a...","SELECT SUM(amount) AS total_amount, MONTH(tran...","SELECT SUM(amount) AS total_amount, STRFTIME('...","SELECT month_id, SUM(ntx_pointx_financial) FRO...","SELECT month_id, SUM(ntx_pointx_financial) FRO...","SELECT month_id, SUM(ntx_pointx_financial) FRO...","SELECT month_id, SUM(ntx_pointx_financial) AS ...","SELECT SUM(mtd1_amt_point_topup_auto_wealth), ...","SELECT SUM(ntx_pointx_financial), month_id FRO...","SELECT month_id, SUM(ntx_pointx_financial) FRO...","SELECT month_id, SUM(ntx_pointx_financial) FRO...","SELECT month_id, SUM(ntx_pointx_financial) FRO..."
1,SELECT SUM(amt_point_topup) FROM pointx_keymat...,SELECT SUM(points) AS total_points\nFROM top_u...,SELECT SUM(points) FROM transactions WHERE tra...,SELECT SUM(points) FROM transactions WHERE tra...,SELECT SUM(amount) FROM financial_transactions...,SELECT SUM(amt_point_topup) FROM pointx_keymat...,SELECT SUM(amt_point_topup) FROM pointx_keymat...,SELECT SUM(amt_point_topup) FROM pointx_keymat...,SELECT SUM(amt_point_topup) FROM pointx_keymat...,SELECT SUM(mtd1_amt_point_topup) FROM pointx_k...,SELECT SUM(amt_point_topup) FROM pointx_keymat...,SELECT SUM(amt_point_topup) FROM pointx_keymat...,SELECT SUM(amt_point_topup) FROM pointx_keymat...,SELECT SUM(amt_point_topup) FROM pointx_keymat...
2,"SELECT month_id, SUM(amt_point_pay) FROM point...","SELECT \n DATE_FORMAT(transaction_date, '%Y...","SELECT MONTH(payment_date) AS month, SUM(point...","SELECT DATE_FORMAT(payment_date, '%Y-%m') AS m...",SELECT SUM(amount) FROM financial_transactions...,"SELECT month_id, SUM(amt_point_pay) FROM point...","SELECT month_id, SUM(amt_point_pay) FROM point...","SELECT month_id, SUM(amt_point_pay) FROM point...","SELECT\n month_id,\n SUM(amt_point_pay) AS t...",SELECT SUM(mtd1_amt_point_pay) FROM pointx_key...,SELECT SUM(amt_point_pay) FROM pointx_keymatri...,SELECT SUM(amt_point_pay) FROM pointx_keymatri...,SELECT SUM(amt_point_pay) FROM pointx_keymatri...,"SELECT month_id, SUM(amt_point_pay) AS total_p..."
3,SELECT AVG(rate_point_per_baht_pay) FROM point...,SELECT AVG(released_points) AS average_rate\nF...,SELECT AVG(released_points) FROM payment_custo...,SELECT AVG(rate) FROM points WHERE customer_ty...,"SELECT SUM(amount) AS total_amount, strftime('...",SELECT AVG(rate_point_per_baht_pay) FROM point...,SELECT AVG(rate_point_per_baht_pay) FROM point...,SELECT AVG(rate_point_per_baht_pay) FROM point...,SELECT rate_point_per_baht_pay FROM pointx_key...,SELECT AVG(rate_point_per_baht_pay) FROM point...,SELECT AVG(rate_point_per_baht_pay_qr_cs) FROM...,SELECT AVG(rate_point_per_baht_pay_weight) FRO...,SELECT AVG(rate_baht_per_point_pay_weight) FRO...,SELECT AVG(rate_baht_per_point_pay) FROM point...
4,"SELECT month_id, AVG(ncust_visit) FROM pointx_...","SELECT \n DATE_FORMAT(visit_date, '%Y-%m') ...",SELECT AVG(number_of_customers)\nFROM visits\n...,SELECT AVG(monthly_visitors) FROM customer_vis...,"SELECT SUM(amount) AS ""Total Amount""\nFROM fin...","SELECT month_id, AVG(ncust_visit) \nFROM point...",SELECT AVG(ncust_visit) FROM pointx_keymatrix_...,"SELECT month_id, AVG(ncust_pointx_visit) FROM ...","SELECT \n\tmonth_id, \n\tAVG(ncust_visit) AS a...","SELECT AVG(mtd1_ncust_visit), AVG(mtd1_ncust_u...",SELECT AVG(mtd1_ncust_pointx_visit) FROM point...,SELECT AVG(mtd1_ncust_pointx_visit) FROM point...,SELECT AVG(mtd1_ncust_pointx_visit) FROM point...,SELECT AVG(mtd1_ncust_pointx_visit) FROM point...


In [9]:
def query_pointx_db(sql_query):
    try:
        conn = sqlite3.connect(f'../src/pointx/database/pointx.db')
        cursor = conn.cursor()
    except:
        conn.close()
        return "CANNOT CONNECT DATABASE"
    try:
        cursor.execute(sql_query)
        results = cursor.fetchall()
    except:
        conn.close()
        return "CANNOT FETCHING DATA"
    conn.close()
    return results

In [19]:
ncols = len(result_file.columns) // 2
ncols

14

In [21]:
sql_df = result_file.iloc[:,:ncols]
sql_df.head()

,Actual SQL,Pure Question DeepSeek,Pure Question GPT3.5,Pure Question GPT4,Pure Question Gemini,Schema Description DeepSeek,Schema Description GPT3.5,Schema Description GPT4,Schema Description Gemini,ChatQ - Only NSQL,ChatQ - DeepSeek,ChatQ - GPT3.5,ChatQ - GPT4,ChatQ - Gemini
0,"SELECT month_id, SUM(ntx_pointx_financial) FRO...","SELECT \n DATE_FORMAT(transaction_date, '%Y...","SELECT MONTH(transaction_date) AS month, SUM(a...","SELECT SUM(amount) AS total_amount, MONTH(tran...","SELECT SUM(amount) AS total_amount, STRFTIME('...","SELECT month_id, SUM(ntx_pointx_financial) FRO...","SELECT month_id, SUM(ntx_pointx_financial) FRO...","SELECT month_id, SUM(ntx_pointx_financial) FRO...","SELECT month_id, SUM(ntx_pointx_financial) AS ...","SELECT SUM(mtd1_amt_point_topup_auto_wealth), ...","SELECT SUM(ntx_pointx_financial), month_id FRO...","SELECT month_id, SUM(ntx_pointx_financial) FRO...","SELECT month_id, SUM(ntx_pointx_financial) FRO...","SELECT month_id, SUM(ntx_pointx_financial) FRO..."
1,SELECT SUM(amt_point_topup) FROM pointx_keymat...,SELECT SUM(points) AS total_points\nFROM top_u...,SELECT SUM(points) FROM transactions WHERE tra...,SELECT SUM(points) FROM transactions WHERE tra...,SELECT SUM(amount) FROM financial_transactions...,SELECT SUM(amt_point_topup) FROM pointx_keymat...,SELECT SUM(amt_point_topup) FROM pointx_keymat...,SELECT SUM(amt_point_topup) FROM pointx_keymat...,SELECT SUM(amt_point_topup) FROM pointx_keymat...,SELECT SUM(mtd1_amt_point_topup) FROM pointx_k...,SELECT SUM(amt_point_topup) FROM pointx_keymat...,SELECT SUM(amt_point_topup) FROM pointx_keymat...,SELECT SUM(amt_point_topup) FROM pointx_keymat...,SELECT SUM(amt_point_topup) FROM pointx_keymat...
2,"SELECT month_id, SUM(amt_point_pay) FROM point...","SELECT \n DATE_FORMAT(transaction_date, '%Y...","SELECT MONTH(payment_date) AS month, SUM(point...","SELECT DATE_FORMAT(payment_date, '%Y-%m') AS m...",SELECT SUM(amount) FROM financial_transactions...,"SELECT month_id, SUM(amt_point_pay) FROM point...","SELECT month_id, SUM(amt_point_pay) FROM point...","SELECT month_id, SUM(amt_point_pay) FROM point...","SELECT\n month_id,\n SUM(amt_point_pay) AS t...",SELECT SUM(mtd1_amt_point_pay) FROM pointx_key...,SELECT SUM(amt_point_pay) FROM pointx_keymatri...,SELECT SUM(amt_point_pay) FROM pointx_keymatri...,SELECT SUM(amt_point_pay) FROM pointx_keymatri...,"SELECT month_id, SUM(amt_point_pay) AS total_p..."
3,SELECT AVG(rate_point_per_baht_pay) FROM point...,SELECT AVG(released_points) AS average_rate\nF...,SELECT AVG(released_points) FROM payment_custo...,SELECT AVG(rate) FROM points WHERE customer_ty...,"SELECT SUM(amount) AS total_amount, strftime('...",SELECT AVG(rate_point_per_baht_pay) FROM point...,SELECT AVG(rate_point_per_baht_pay) FROM point...,SELECT AVG(rate_point_per_baht_pay) FROM point...,SELECT rate_point_per_baht_pay FROM pointx_key...,SELECT AVG(rate_point_per_baht_pay) FROM point...,SELECT AVG(rate_point_per_baht_pay_qr_cs) FROM...,SELECT AVG(rate_point_per_baht_pay_weight) FRO...,SELECT AVG(rate_baht_per_point_pay_weight) FRO...,SELECT AVG(rate_baht_per_point_pay) FROM point...
4,"SELECT month_id, AVG(ncust_visit) FROM pointx_...","SELECT \n DATE_FORMAT(visit_date, '%Y-%m') ...",SELECT AVG(number_of_customers)\nFROM visits\n...,SELECT AVG(monthly_visitors) FROM customer_vis...,"SELECT SUM(amount) AS ""Total Amount""\nFROM fin...","SELECT month_id, AVG(ncust_visit) \nFROM point...",SELECT AVG(ncust_visit) FROM pointx_keymatrix_...,"SELECT month_id, AVG(ncust_pointx_visit) FROM ...","SELECT \n\tmonth_id, \n\tAVG(ncust_visit) AS a...","SELECT AVG(mtd1_ncust_visit), AVG(mtd1_ncust_u...",SELECT AVG(mtd1_ncust_pointx_visit) FROM point...,SELECT AVG(mtd1_ncust_pointx_visit) FROM point...,SELECT AVG(mtd1_ncust_pointx_visit) FROM point...,SELECT AVG(mtd1_ncust_pointx_visit) FROM point...


In [24]:
result_df = result_file.iloc[:,ncols:]
result_df.head()

,Actual SQL result,Pure Question DeepSeek result,Pure Question GPT3.5 result,Pure Question GPT4 result,Pure Question Gemini result,Schema Description DeepSeek result,Schema Description GPT3.5 result,Schema Description GPT4 result,Schema Description Gemini result,ChatQ - NSQL result,ChatQ - DeepSeek result,ChatQ - GPT3.5 result,ChatQ - GPT4 result,ChatQ - Gemini result
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
for sql_col, result_col in zip(sql_df.columns, result_df.columns):
    result_df[result_col] = sql_df[sql_col].apply(query_pointx_db)

result_df.to_excel('results/temp_experiments.xlsx', index=False)

In [57]:
query = "SELECT customer_type, SUM(ntx) FROM pointx_cust_mly GROUP BY customer_type;"

print(query_pointx_db(query))

[('Easy', 290), ('GUEST', 0), ('Non Easy', 56)]


# Structure by masking